In [1]:
from config import Config
from utils import set_seeds
import random
from dataset import DataProcessor, get_loaders

args = Config()
set_seeds(args.seed)
trained_model_path = ''

processor = DataProcessor(args)
processor.set_data_args()

''' Get Loaders '''
loader_dict = get_loaders(args, processor.tech_dict, processor.close_dict, processor.date_dict, processor.cov_dict)

/home/dlw/anaconda3/envs/finrl/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
100%|██████████| 4007/4007 [00:30<00:00, 130.67it/s]


In [2]:
import torch
import copy
import model as model_factory
from test import eval_model, eval_model_with_costv2, eval_model_with_action_optimize
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F
from utils import load_torch_file

# Set Running Device
device = torch.device(args.device)
# TODO: GET DIFFERENT MODEL
model = getattr(model_factory, args.model_name)(args).to(device)

load_torch_file(model, trained_model_path)

/home/dlw/anaconda3/envs/finrl/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
import json
import torch

# TO BE FILLED
best_predict_return = ''
prediction_model = ''
with open(best_predict_return) as f:
    best_predict_return_results = json.load(f)
best_return_rate = torch.tensor(best_predict_return_results['expected_return'], dtype=torch.float)
best_return_rate = best_return_rate.to(device)

In [4]:
def get_pretrained_actions(loader_dict, tag='valid'):
    # Get Validation
    valid_loader = loader_dict[tag]
    valid_loader, _, _, _, _ = valid_loader
    model.eval()
    with torch.no_grad():
        all_action_valid = []
        all_diff_valid = []
        all_yr_valid = []
        all_cov_valid = []
        all_hn_valid = []
        for batch in valid_loader:
            state, diff, idx, assets_cov = [item.to(device) for item in batch]
            action, y_r, hn = model.forward_with_hiddenstate(state, assets_cov)
            # action = action.softmax(-1)
            all_action_valid.append(action)
            all_diff_valid.append(diff)
            all_yr_valid.append(y_r)
            all_hn_valid.append(hn)
            all_cov_valid.append(assets_cov)
    all_action_valid = torch.cat(all_action_valid)
    all_action_valid_softmax = all_action_valid.softmax(-1)
    all_diff_valid = torch.cat(all_diff_valid)
    all_yr_valid = torch.cat(all_yr_valid)
    all_hn_valid = torch.cat(all_hn_valid)
    all_cov_valid = torch.cat(all_cov_valid)
    return all_action_valid, all_action_valid_softmax, all_diff_valid, all_yr_valid, all_hn_valid, all_cov_valid

all_action_test, all_action_test_softmax, all_diff_test, all_yr_test, all_hn_test, all_cov_test = get_pretrained_actions(loader_dict, tag='test')

In [8]:
all_action_test.shape

torch.Size([232, 28])

In [5]:
# TYPE 4 Interpolation
from pypfopt.efficient_frontier import EfficientFrontier
import cvxpy
import torch
import numpy as np

def get_minvar_actions(all_cov_valid, all_diff_valid):
    total_assets = 1
    total_assets_list_minvar_valid = [total_assets]
    weights_list_valid = []
    for idx in range(0, len(all_cov_valid)):
        try:
            ef_min_var = EfficientFrontier(None, all_cov_valid[idx].detach().cpu().numpy())
            raw_weights_min_var = ef_min_var.min_volatility()
            cleaned_weights_min_var = ef_min_var.clean_weights()
            weights = np.array([element for element in cleaned_weights_min_var.values()])
            total_assets = ((weights * (all_diff_valid[idx].cpu().numpy() - 1)).sum() + 1) * total_assets
        except:
            try:
                total_assets = ((weights * (all_diff_valid[idx].cpu().numpy() - 1)).sum() + 1) * total_assets
            except:
                # if weights is None
                continue
        total_assets_list_minvar_valid.append(total_assets)
        weights_list_valid.append(torch.tensor(weights, dtype=torch.float))
    # check length
    if len(weights_list_valid) != len(all_cov_valid):
        weights_list_valid = (len(all_cov_valid) - len(weights_list_valid)) * [weights_list_valid[0]] + weights_list_valid
    actions_minvar_valid = torch.stack(weights_list_valid).to(device)
    return actions_minvar_valid, total_assets_list_minvar_valid

actions_minvar_test, total_assets_list_minvar_test = get_minvar_actions(all_cov_test, all_diff_test)

In [62]:
''' Portfolio Improvement + Return Rate'''
import torch
import torch.nn as nn
from torch.utils.data.dataset import TensorDataset
from torch.utils.data.dataloader import DataLoader

# DOW30
# given_risks = [1e-5, 2e-5, 3e-5, 4e-5, 5e-5, 6e-5, 7e-5, 8e-5, 9e-5, 10e-5]
# NAS100
given_risks = [1e-4, 2e-4, 3e-4, 4e-4, 5e-4, 6e-4, 7e-4, 8e-4, 9e-4, 10e-4]
for given_risk in given_risks:
    set_seeds(args.seed)

    import copy

    all_action_test_adjust = copy.deepcopy(all_action_test)
    all_action_test_adjust.requires_grad = True
    optimizer = torch.optim.AdamW([all_action_test_adjust], lr=0.1, weight_decay=0.)

    total_assets_list_test_all = []

    for i in range(30):
        all_action_test_adjust_softmax = all_action_test_adjust.softmax(-1)

        aAa = all_action_test_adjust_softmax.unsqueeze(1) @ all_cov_test @ all_action_test_adjust_softmax.unsqueeze(-1)
        bAb = actions_minvar_test.unsqueeze(1) @ all_cov_test @ actions_minvar_test.unsqueeze(-1)
        aAb = all_action_test_adjust_softmax.unsqueeze(1) @ all_cov_test @ actions_minvar_test.unsqueeze(-1)

        a = aAa - 2 * aAb + bAb
        b = 2 * aAb - 2 * aAa
        c = aAa - given_risk
        after_square = torch.sqrt(F.relu(b * b - 4 * a * c))
        w1 = (-b + after_square) / 2 / a
        w2 = (-b - after_square) / 2 / a
        w3 = torch.where((w1 <= 1) & (w1 >=0), w1, 0) + torch.where((w2 <= 1) & (w2 >=0), w2, 0)
        w3 = torch.where(w3 <= 1, w3, 1)
        filtered_w2 = w3

        reweight_action = ((1 - filtered_w2).squeeze(-1) * all_action_test_adjust_softmax + filtered_w2.squeeze(-1) * actions_minvar_test)
        return_rate = (best_return_rate * reweight_action).sum(-1)

        com_loss = filtered_w2.squeeze(-1) - 100 * return_rate
        loss = torch.where(((aAa >= given_risk) & (bAb <= given_risk)).squeeze(-1).squeeze(-1), com_loss, 0)
        loss = loss.sum()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item())

        total_assets_list = [1] + ((reweight_action * (all_diff_test - 1)).sum(-1) + 1).cumprod(-1).detach().cpu().numpy().tolist()
        total_assets_list_test_all.append(total_assets_list)

    ''' Test '''
    from pyfolio import timeseries
    import json

    def get_performance(total_assets_list, tag='interpolation'):
        period_return = pd.Series(np.array(total_assets_list)).pct_change(1).dropna()
        perf_stats_all = timeseries.perf_stats(period_return)
        perf_stats_all_dict = perf_stats_all.to_dict()
        perf_stats_all_dict['cumulative wealth'] = total_assets_list
        result_save_dir = Path(args.result_save_dir) / args.dataset
        if not result_save_dir.exists():
            result_save_dir.mkdir()
        result_save_path = result_save_dir / 'riskcontrol+{}+return_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.json'.format(prediction_model, tag, given_risk, args.model_name, args.net_dims, args.dataset, args.optimizer, args.batch_size, args.target, args.trans_cost, args.window, args.alpha_downside, args.action_constraint, args.action_constraint_weight, args.mse_alpha, args.ranking_alpha)
        with open(result_save_path, 'w') as f:
            json.dump(perf_stats_all_dict, f, indent=4)
        return perf_stats_all

    total_assets_list_test = []
    risks = []
    reweight_action_list = []
    with torch.no_grad():
        aAa = all_action_test_softmax.unsqueeze(1) @ all_cov_test @ all_action_test_softmax.unsqueeze(-1)
        bAb = actions_minvar_test.unsqueeze(1) @ all_cov_test @ actions_minvar_test.unsqueeze(-1)
        aAb = all_action_test_softmax.unsqueeze(1) @ all_cov_test @ actions_minvar_test.unsqueeze(-1)
        # given risk should not less than min_risk
        min_risk = torch.where(bAb > given_risk, bAb, given_risk)
        a = aAa - 2 * aAb + bAb
        b = 2 * aAb - 2 * aAa
        c = aAa - min_risk
        after_square = torch.sqrt(F.relu(b * b - 4 * a * c))
        w1 = (-b + after_square) / 2 / a
        w2 = (-b - after_square) / 2 / a
        w3 = torch.where((w1 <= 1) & (w1 >=0), w1, 0) + torch.where((w2 <= 1) & (w2 >=0), w2, 0)
        w3 = torch.where(w3<=1, w3, 1)
        filtered_w2 = w3
        ori_reweight_action = ((1 - filtered_w2).squeeze(-1) * all_action_test_softmax + filtered_w2.squeeze(-1) * actions_minvar_test)
        total_assets_list = [1] + ((ori_reweight_action * (all_diff_test - 1)).sum(-1) + 1).cumprod(-1).detach().cpu().numpy().tolist()
        total_assets_list_test.append(total_assets_list)
        risks.append(ori_reweight_action.unsqueeze(1) @ all_cov_test @ ori_reweight_action.unsqueeze(-1))
        reweight_action_list.append(ori_reweight_action)
        print(get_performance(total_assets_list, tag='interpolation').to_dict())

        adjusted_action = all_action_test_adjust.softmax(-1)
        aAa = adjusted_action.unsqueeze(1) @ all_cov_test @ adjusted_action.unsqueeze(-1)
        bAb = actions_minvar_test.unsqueeze(1) @ all_cov_test @ actions_minvar_test.unsqueeze(-1)
        aAb = adjusted_action.unsqueeze(1) @ all_cov_test @ actions_minvar_test.unsqueeze(-1)
        a = aAa - 2 * aAb + bAb
        b = 2 * aAb - 2 * aAa
        c = aAa - min_risk
        after_square = torch.sqrt(F.relu(b * b - 4 * a * c))
        w1 = (-b + after_square) / 2 / a
        w2 = (-b - after_square) / 2 / a
        w3 = torch.where((w1 <= 1) & (w1 >=0), w1, 0) + torch.where((w2 <= 1) & (w2 >=0), w2, 0)
        w3 = torch.where(w3<=1, w3, 1)
        filtered_w2 = w3
        reweight_action = ((1 - filtered_w2).squeeze(-1) * adjusted_action + filtered_w2.squeeze(-1) * actions_minvar_test)

        optimized_risks = reweight_action.unsqueeze(1) @ all_cov_test @ reweight_action.unsqueeze(-1)

        total_assets_list = [1] + ((reweight_action * (all_diff_test - 1)).sum(-1) + 1).cumprod(-1).detach().cpu().numpy().tolist()
        total_assets_list_test.append(total_assets_list)
        risks.append(reweight_action.unsqueeze(1) @ all_cov_test @ reweight_action.unsqueeze(-1))
        reweight_action_list.append(reweight_action)
        print(get_performance(total_assets_list, tag='modelfinetune').to_dict())

21759.0078125
21632.0859375
21513.955078125
21404.9375
21304.951171875
21213.328125
21128.91015625
21050.29296875
20975.84375
20903.99609375
20833.6484375
20764.24609375
20695.291015625
20625.28125
20551.47265625
20470.96875
20381.48828125
20281.06640625
20167.7890625
20039.97265625
19898.189453125
19741.171875
19569.76171875
19385.73046875
19190.521484375
18984.318359375
18765.3046875
18531.376953125
18280.828125
18009.953125
{'Annual return': 0.6386031219082493, 'Cumulative returns': 0.5787128210067762, 'Annual volatility': 0.4203191091477773, 'Sharpe ratio': 1.3813230177683937, 'Calmar ratio': 3.0052908789724215, 'Stability': 0.7210744597707792, 'Max drawdown': -0.21249294914394526, 'Omega ratio': 1.3176463566675216, 'Sortino ratio': 2.3383818415587356, 'Skew': 1.1545025545906522, 'Kurtosis': 7.8784044161567515, 'Tail ratio': 1.2649295911013492, 'Daily value at risk': -0.05065127598748039}
{'Annual return': 0.5940604850218252, 'Cumulative returns': 0.5389927625656123, 'Annual volati